In [11]:
from sympy import Symbol, diff, hessian
import numpy as np 

In [2]:
n_variaveis = 2

x_1 = Symbol('x_1')
x_2 = Symbol('x_2')

funcao = (x_1**2 + x_2 - 11)**2 + (x_1 + x_2**2 - 7)**2
funcao

In [9]:
def calcula_gradiente(funcao, variaveis):
    return [diff(funcao, variavel) for variavel in variaveis]

def calcula_hessiana(funcao, variaveis):
  return hessian(funcao, variaveis)

In [4]:
gradiente = calcula_gradiente(funcao, [x_1, x_2])
gradiente

In [12]:
hessiana = calcula_hessiana(funcao, [x_1, x_2])
hessiana

In [35]:
def substitui_variaveis_funcao(funcao, variaveis, ponto):
    return funcao.subs(dict(zip(variaveis, ponto)))

def substitui_variaveis_gradiente(gradiente, variaveis, ponto):
    return np.array([gradiente_i.subs(dict(zip(variaveis, ponto))) for gradiente_i in gradiente]).astype(np.float64)

def substitui_variaveis_hessiana(hessiana, variaveis, ponto):
    return np.array(hessiana.subs(dict(zip(variaveis, ponto)))).astype(np.float64)

![Armijo's Rule](./img/alg_busca_armijo.png)

In [ ]:
def busca_de_armijo(funcao: any, gradiente: any, variaveis: list, ponto_inicial: np.array, d: np.array, gama: float, eta: float):
  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - d: direção de descida
    - gama: fator de redução do passo
    - eta: fator de ???
  
  Retorna:
    - t: passo ótimo
    - iteracoes: número de iterações
  """
  t = 1
  iteracoes = 1
  while substitui_variaveis_funcao(funcao, variaveis, ponto_inicial + t*d) > substitui_variaveis_funcao(funcao, variaveis, ponto_inicial) + eta*t*np.dot(substitui_variaveis_gradiente(gradiente, variaveis, ponto_inicial), d):
    t = gama*t
    iteracoes += 1

  return t, iteracoes

<h2>Método do Gradiente</h2>
<img src="./img/alg_metodo_gradiente.png" width="500">


In [ ]:
def metodo_do_gradiente(
    funcao: any,
    gradiente: any, 
    variaveis: list, 
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000, 
    valor_minimo: float = 1e-20):
  
  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução
  
  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial
  while np.linalg.norm(substitui_variaveis_gradiente(gradiente, variaveis, ponto)) > valor_minimo:
    d = -substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp
    ponto = ponto + t*d
    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1
  
  return ponto, iteracoes, iteracoes_armijo

<h2>Método de Newton</h2>
<img src="./img/alg_metodo_newton.png" width="500">

In [ ]:
def metodo_de_newton(
    funcao: any,
    gradiente: any,
    hessiana: any,
    variaveis: list, 
    ponto_inicial: np.array,
    gama: float,
    eta: float,
    maximo_iteracoes: int = 1000, 
    valor_minimo: float = 1e-20):
  
  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - laplaciano: laplaciano da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto_inicial: ponto inicial
    - gama: fator de redução do passo
    - eta: fator de ???
    - maximo_iteracoes: número máximo de iterações
    - valor_minimo: valor mínimo da função para parar a execução
  
  Retorna:
    - ponto: ponto ótimo
    - iteracoes: número de iterações
    - iteracoes_armijo: número de iterações da busca de armijo
  """

  iteracoes = 0
  iteracoes_armijo = 0
  ponto = ponto_inicial

  while np.linalg.norm(substitui_variaveis_gradiente(ponto)) > valor_minimo:
    d = -np.linalg.inv(substitui_variaveis_hessiana(hessiana, variaveis, ponto)) @ substitui_variaveis_gradiente(gradiente, variaveis, ponto)
    t, iteracoes_armijo_tmp = busca_de_armijo(funcao, gradiente, variaveis, ponto, d, gama, eta)
    iteracoes_armijo += iteracoes_armijo_tmp
    ponto = ponto + t*d
    if iteracoes > maximo_iteracoes:
      print('Número máximo de iterações atingido')
      break
    iteracoes += 1

  return ponto, iteracoes, iteracoes_armijo

In [ ]:
from sympy import symbols

# Define the variables x1 and x2
x1, x2 = symbols('x1 x2')

# Define the function
f_x = (1/2) * (x1 - 2)**2 + (x2 - 1)**2
f_x
print(f_x)
# Define the gradient
grad_f_x = [f_x.diff(x1), f_x.diff(x2)]
print(grad_f_x)



In [ ]:
from sympy import symbols

# Define the variables x1 and x2
variaveis = list(symbols('x1 x2'))
print(variaveis)
x1, x2 = variaveis

# Define the function
f_x = (1/2) * (x1 - 2)**2 + (x2 - 1)**2
f_x
print(f_x)

# Define the gradient
grad_f_x = calcula_gradiente(f_x, variaveis)
print(grad_f_x)



In [ ]:
ponto_inicial = np.array([1, 0])
d = np.array([3, 1])
eta = 0.25
gama = 0.8

passo_otimo, n_iteracoes = busca_de_armijo(f_x, grad_f_x, variaveis, ponto_inicial, d, gama, eta)